In [26]:
import csv
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import Dataset
from torch.utils.data import DataLoader



# podaci u datasetu:
# starost pacijenta
# godina operacije
# aksilarni limfni čvorovi
# klase

In [27]:
# Definisanje klase za opis skupa podataka.
class HabermanDataset(Dataset):
    def __init__(self, path=None):
        super().__init__()
        # Skladiste skupa podataka.
        self.data = []
        
        # Mapiranje imena klasa na numericke vrednosti.
        self.label_index = {
            "Survived-more": 0,
            "Survived-less": 1
        }
        
        # Ucitavanje podataka sa zadate putanje.
        self._load(path)
        
    def _load(self, path):
        with open(path) as fp:
            csv_data = csv.reader(fp)
            for row in csv_data:
                if len(row) > 0:
                    # Parsiranje i skladistenje reda csv datoteke.
                       self.data.append((np.array(np.array(list(map(lambda x: float(x), row[:-1]))), dtype=np.float32), self.label_index[row[-1]]))
        
    def __getitem__(self, index):
        # Ova magicna metoda mora biti implementirana.
        # Predstavlja implementaciju preklapanja operatora
        # indeksiranja. Kao argument prima indeks, a kao povratnu
        # vrednost vraca podatak na zadatom indeksu.
        return self.data[index]
    
    def __len__(self):
        # Ova magicna metoda mora biti implementirana.
        # Povratna vrednost ove metode predstavlja ukupan broj
        # pojava u skupu podataka.
        return len(self.data)

In [28]:
ds = HabermanDataset("haberman.data")
test_loader = DataLoader(ds, shuffle=True, batch_size=60)
train_loader = DataLoader(ds, shuffle=True, batch_size=60)

In [29]:
class Net(nn.Module):
    def __init__(self):
        
        #Definisanje neuronske mreze
        # 3 potpuno povezana sloja
        # broj ulaznih kanala jednak broju izlaznih prethodnog sloja
        # broj izlaznih kanala poslednjeg sloja je broj klasa
        
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3, 50)
        self.fc2 = nn.Linear(50, 20)
        self.fc3 = nn.Linear(20, 2)

    def forward(self, x):
        
        #forward pass
        # prosledjivanje ulaza i racunanje izlaza za svaki sloj od prvog do poslednjeg
        
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [43]:
net = Net()

# instanciranje kriterijuma optimizacije

criterion = nn.CrossEntropyLoss()


# Instanciranje optimizacionog algoritma. U ovom slucaju
# stochastic gradient descent. Optimizacioni algoritam
# optimizuje parametre neuronske mreze, odnosno tezine
# na ulazima neurona, learning rate i moment podeseni
# na 0.001 i 0.9 kako bi se osigurala stabilna konvergencija.

#Stochastic gradient descent is a method to find the optimal parameter configuration for a machine learning algorithm.
#It iteratively makes small adjustments to a machine learning network configuration to decrease the error of the network.

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


#Proces obucavanja

#One Epoch is when an entire dataset is passed forward and backward through the neural network only once

for epoch in range(5):
    running_loss = 0.0 # Pocetna greska u toku epohe
    
    # U svakoj epohi vrsi se obucavanje nad citavim obucavajucim
    # skupom podataka. Pomocu data loader-a iz skupa podataka
    # se izdvajaju obucavajuci podskupovi predefinisane velicine.
    # Takodje prilikom izdvjanja podskupova vrsi se i primena
    # prethodno definisanih transformacija.
     

    for i, data in enumerate(train_loader):
        
        # Za svaku instancu obucavajuceg skupa se
        # dobavljaju vektor osobina i labela.
        
        inputs, labels = data
        
        # Resetuju se gradijenti akumulirani prilikom
        # prethodnih izvrsenja optimizacionog algoritma.        
        optimizer.zero_grad()
        
        # Vrsi se izracunavanje izlaza neuronske mreze.
        outputs = net(inputs)
        
        # Na osnovu kriterijuma optimizacije vrsi se racunanje greske.
        loss = criterion(outputs, labels)
        
        # Greska se propagira nazad kroz mrezu.
        #backward pass refers to process of counting changes in weights (de facto learning), 
        #using gradient descent algorithm (or similar)
        loss.backward()
        
        # Vrsi se optimizacija parametara.
        optimizer.step()
        
        # Uvecava se ukupna greska tokom jedne epohe.
        running_loss += loss.item()
        
        #Ispis greske
        if i % 5 == 4:
            print('%d. epoch loss: %.3f' % (epoch + 1, running_loss / 5))
            running_loss = 0.0

print("Obucavanje zavrseno.")

1. epoch loss: 3.501
2. epoch loss: 3.134
3. epoch loss: 1.591
4. epoch loss: 0.918
5. epoch loss: 0.649
Obucavanje zavrseno.


In [44]:
total = 0 #Ukupan broj primera
correct = 0 # Broj tacno klasifikovanih primera


# Tokom upotrebe neuronske mreze, nakon sto je obucena,
# nema potrebe racunati gradijente.
with torch.no_grad():
    for data in test_loader:
         # Dobavljanje podskupa iz obucavajuceg skupa podataka.
        features, labels = data
        
        # Racunanje izlaza neuronske mreze.
        outputs = net(features)
        
        # Dobavljanje indeksa izlaza sa najjacom pobudom.
        _, predicted = torch.max(outputs.data, 1)
        
        # Uvecavanje broja ukupno obradjenih primera.
        total += labels.size(0)
        
        # Ukoliko je indeks labele jednak indeksu izlaza neuronska
        # mreza je izvrsila tacnu predikciju.
        correct += (predicted == labels).sum().item()

print("Tacnost: {:.3f}%".format(100 * correct / total))

Tacnost: 72.876%


In [7]:
#duzina dataseta
ds.__len__()

306

In [8]:
#prvi podatak
ds.__getitem__(0)

(array([30., 64.,  3.], dtype=float32), 0)

In [67]:
#poslednji
ds.__getitem__(305)

(array([83., 58.,  4.], dtype=float32), 1)